<a href="https://colab.research.google.com/github/jbaek1/d2lbrightspaceamplify/blob/main/Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [1]:
!pip install tqdm pandas
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [3]:
#Describe your answer below.
#Answer 1: This code is downloading a list of symbols on NASDAQ with | as the column separator, then loads all the data in to a Pandas DF called "data".
# Then the "data" is cleaned into a new dataframe called "data_clean" that only contains symbols with the "Test_Issue" value being "N", and keeps just the NASDAQ symbols as a python list and converts to a set.
#Next, the same is done but for other symbols not on NASDAQ, it seems that the goal was to combine NASDAQ symbols with non-NASDAQ symbols, but this is not being done correctly. Instead, the old symbols set is being overwritten by the symbols traded elsewhere.

In [4]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2026-01-11 21:31:07--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/pm4vi37h059bx0du3ecj8/stocks.parquet?rlkey=8k31klc1tlf8bk4ocxpzbgm9c&dl=1 [following]
--2026-01-11 21:31:07--  https://www.dropbox.com/scl/fi/pm4vi37h059bx0du3ecj8/stocks.parquet?rlkey=8k31klc1tlf8bk4ocxpzbgm9c&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca249e7d43d0eb834e18f2338e8.dl.dropboxusercontent.com/cd/0/inline/C4yrTroWJ8hcpEiWdc_o-ZhfUYb51p57OtpgOMyE8tx0LaE7KijflZP_lqB-AUlnn_eyriXopzw7a7yw3eQ-3Byce7qAcwe_Z-wOE3mCL6SAn_HQw2P3Wp1mu3IldjK70wAcAfOQPyOqUk2vbVuxgsRV/file?dl=1# [following]
--2026-01-11 21:31:08--  https://uca249e7d43d0eb834e18f2338e8.dl.dropboxuse

In [5]:
!ls stocks.parquet -l #downloaded file

-rw-rw-r-- 1 ubuntu ubuntu 678640589 Jan 11 21:31 stocks.parquet


In [6]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list.

In [7]:
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = symbols.union(set(data_clean['NASDAQ Symbol'].tolist()))

In [8]:
data=pd.read_parquet('stocks.parquet')

In [9]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock


,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [10]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

data['date'] = pd.to_datetime(data['date'])

# 2. Filter data once
cutoff = pd.Timestamp("2021-01-01")
data_cut = data[data['date'] <= cutoff]

# 3. Get the start and end dates for each symbol
# This is much faster than sorting the whole dataframe
stats = data_cut.groupby('symbol')['date'].agg(['min', 'max']).reset_index()

merged = stats.merge(data_cut[['symbol', 'date', 'adj_close']],
                     left_on=['symbol', 'min'], right_on=['symbol', 'date'])
merged = merged.rename(columns={'adj_close': 'adj_close_start', 'min': 'date_start'}).drop(columns='date')

merged = merged.merge(data_cut[['symbol', 'date', 'adj_close']],
                     left_on=['symbol', 'max'], right_on=['symbol', 'date'])
merged = merged.rename(columns={'adj_close': 'adj_close_end', 'max': 'date_end'}).drop(columns='date')

# 5. Calculate Z and Return
merged['z'] = (merged['date_end'] - merged['date_start']).dt.days

# Avoid division by zero and calculate return
merged = merged[merged['z'] > 0].copy()
merged['return'] = merged['adj_close_end'] / merged['adj_close_start'] / merged['z']

# 6. Get Top 20
t_20 = merged.sort_values('return', ascending=False).head(20)[['symbol', 'return']]


In [12]:
t_20

,symbol,return
8014,VSDA,inf
3156,GIGB,inf
7765,USOI,inf
4632,LRFC,inf
7571,TUSA,inf
4252,JPST,inf
2800,FLQL,inf
2802,FLQS,inf
2801,FLQM,inf
8322,XT,inf


In [13]:
# 1. Ensure the date column is in datetime format
data['date'] = pd.to_datetime(data['date'])

# 2. Filter data for the end point (2021-01-01)
# Note: If 2021-01-01 isn't a trading day, we look for the last available date up to that point
end_date = '2021-01-01'
data_filtered = data[data['date'] <= end_date]

# 3. Create a summary dataframe grouped by symbol
performance = data_filtered.groupby('symbol').agg(
    earliest_date=('date', 'min'),
    latest_date=('date', 'max'),
    X=('adj_close', 'first'),  # adj_close at the earliest date
    Y=('adj_close', 'last')    # adj_close at the latest date (closest to 2021-01-01)
)

# 4. Calculate Z (number of days elapsed)
performance['Z'] = (performance['latest_date'] - performance['earliest_date']).dt.days

# 5. Calculate the Return (Y / X / Z)
# Note: We ensure Z > 0 to avoid division by zero
performance = performance[performance['Z'] > 0]
performance['return_metric'] = performance['Y'] / performance['X'] / performance['Z']

# 6. Sort and get top 20
top_20_performers = performance.sort_values(by='return_metric', ascending=False).head(20)

print(top_20_performers[['return_metric']])

        return_metric
symbol               
VSDA              inf
FLQL              inf
FLQS              inf
USOI              inf
GIGB              inf
LRFC              inf
XT                inf
FLQM              inf
TUSA              inf
JPST              inf
PRTA              inf
NVO      2.078044e+06
NKE      2.540825e+03
DCI      3.768860e+02
TJX      1.923844e+02
WTRG     6.511958e+00
TTC      5.989074e+00
USB      3.342177e+00
MCD      2.686572e+00
PII      2.294361e+00


In [14]:
performance['Z'] = (performance['latest_date'] - performance['earliest_date']).dt.days

clean_performance = performance[(performance['X'] > 0) & (performance['Z'] > 0)].copy()

# 3. Calculate Return
clean_performance['return_metric'] = clean_performance['Y'] / clean_performance['X'] / clean_performance['Z']

# 4. Sort for top performers
top_20 = clean_performance.sort_values(by='return_metric', ascending=False).head(20)

In [15]:
top_20

,earliest_date,latest_date,X,Y,Z,return_metric
symbol,,,,,,
NVO,1981-04-30,2020-12-31,2.275794e-09,68.526123,14490,2.078044e+06
NKE,1980-12-02,2020-12-31,3.775067e-06,140.414154,14639,2.540825e+03
DCI,1980-03-17,2020-12-31,9.812100e-06,55.097141,14899,3.768860e+02
TJX,1987-06-26,2020-12-31,2.857077e-05,67.289032,12242,1.923844e+02
WTRG,1980-03-17,2020-12-31,4.768529e-04,46.265060,14899,6.511958e+00
TTC,1980-03-17,2020-12-31,1.051490e-03,93.825699,14899,5.989074e+00
USB,1973-05-03,2020-12-31,7.828808e-04,45.551102,17409,3.342177e+00
MCD,1966-07-05,2020-12-31,3.923203e-03,209.776978,19903,2.686572e+00
PII,1987-09-16,2020-12-31,3.345692e-03,93.342911,12160,2.294361e+00


In [11]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS